# ![Heroes of Pymoli](Images/Fantasy.jpg)

Congratulations! After a lot of hard work in the data munging mines, you've landed a job as Lead Analyst for an independent gaming company. You've been assigned the task of analyzing the data for their most recent fantasy game Heroes of Pymoli. 

Like many others in its genre, the game is free-to-play, but players are encouraged to purchase optional items that enhance their playing experience. As a first task, the company would like you to generate a report that breaks down the game's purchasing data into meaningful insights.

In [1]:
import pandas as pd
import json


In [2]:
filename = 'purchase_data2.json'
read_json = pd.read_json(filename)
raw_data = pd.DataFrame(read_json)
raw_data.head()


,Age,Gender,Item ID,Item Name,Price,SN
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35
1,21,Male,12,Dawne,3.36,Aidaira26
2,17,Male,5,Putrid Fan,2.63,Irim47
3,17,Male,123,Twilight's Carver,2.55,Irith83
4,22,Male,154,Feral Katana,4.11,Philodil43


# **Player Count**

* Total Number of Players

In [3]:
total_players = raw_data['SN'].nunique()
unique_items = raw_data['Item Name'].nunique()
total_purchases = raw_data.count().max()



74

# **Gender Demographics**

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

**Purchasing Analysis (Gender)** 

* The below each broken by gender
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Normalized Totals

In [6]:
#raw_data['Gender'].unique()       #check genders for consistency

male_data = raw_data.loc[raw_data['Gender'] == 'Male']
#male_data['Gender'].unique()      #check for successful filter, should be 'Male'

male_count = male_data['SN'].nunique()
male_percentage = male_count / total_players

male_purchases= male_data.count().max()
male_total_purchase_value = male_data['Price'].sum()
average_male_price = male_total_purchase_value / male_purchases

male_dict = {
    'Gender': 'Male',
    'Players': male_count, 
    'Percentage': male_percentage, 
    'Number_Purchases': male_purchases,
    'Average_Price': average_male_price,
    'Total_Value': male_total_purchase_value,
    'Normalized_Value': male_total_purchase_value / male_count
             }

#print(f' # men {male_count} \n % men {male_percentage} \n male purchases {male_purchases} \n male purchase value {male_total_purchase_value} \naverage male purchase value {average_male_price}')

In [7]:
female_data = raw_data.loc[raw_data['Gender'] == 'Female']
#female_data['Gender'].unique()    #check for successful filter, should be 'Female'

female_count = female_data['SN'].nunique()
female_percentage = female_count / total_players

female_purchases= female_data.count().max()
female_total_purchase_value = female_data['Price'].sum()
average_female_price = female_total_purchase_value / female_purchases

female_dict = {
    'Gender': 'Female',
    'Players': female_count, 
    'Percentage': female_percentage, 
    'Number_Purchases': female_purchases,
    'Average_Price': average_female_price,
    'Total_Value': female_total_purchase_value,
    'Normalized_Value': female_total_purchase_value / female_count
             }

#print(f' # female {female_count} \n % female {female_percentage} \n female purchases {female_purchases} \n female purchase value {female_total_purchase_value} \n average female purchase value {average_female_price}')

In [8]:
other_data = raw_data.loc[(raw_data['Gender'] != 'Male') & (raw_data['Gender'] != 'Female')]
#other_data['Gender'].unique()     #check for successful filter, should not be 'Male' or 'Female'

other_count = other_data['SN'].nunique()
other_percentage = other_count / total_players

other_purchases = other_data.count().max()
other_total_purchase_value = other_data['Price'].sum()
average_other_price = other_total_purchase_value / other_purchases

other_dict = {
    'Gender': 'Other',
    'Players': other_count, 
    'Percentage': other_percentage, 
    'Number_Purchases': other_purchases,
    'Average_Price': average_other_price,
    'Total_Value': other_total_purchase_value,
    'Normalized_Value': other_total_purchase_value / other_count
             }

#print(f' # other {other_count} \n % other {other_percentage} \n other purchases {other_purchases} \n other purchase value {other_total_purchase_value} \n average other price {average_other_price}')

In [9]:
columns = [
    male_dict,
    female_dict,
    other_dict
    ]
gender_df = pd.DataFrame(columns)
gender_df.set_index('Gender')

,Average_Price,Normalized_Value,Number_Purchases,Percentage,Players,Total_Value
Gender,,,,,,
Male,2.884375,3.076667,64,0.810811,60,184.60
Female,3.183077,3.183077,13,0.175676,13,41.38
Other,2.120000,2.120000,1,0.013514,1,2.12


# **Age Demographics**

* The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.) 
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value
  * Normalized Totals

In [10]:
print(f"{raw_data['Age'].min()} - {raw_data['Age'].max()} years")

7 - 40 years


In [11]:
bins = [6,10,14,18,22,26,30,34,38,42,46]
group_labels = ['06-10', '10-14', '14-18', '18-22','22-26','26-30','30-34','34-38','38-42','42-46']
age_binned = pd.cut(raw_data['Age'], bins,labels = group_labels)

In [12]:
raw_data['Age Group']= age_binned
raw_data.head()

,Age,Gender,Item ID,Item Name,Price,SN,Age Group
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35,18-22
1,21,Male,12,Dawne,3.36,Aidaira26,18-22
2,17,Male,5,Putrid Fan,2.63,Irim47,14-18
3,17,Male,123,Twilight's Carver,2.55,Irith83,14-18
4,22,Male,154,Feral Katana,4.11,Philodil43,18-22


In [13]:
grouped = raw_data.groupby('Age Group')

columns = {
    'Proportion': grouped['SN'].nunique() / total_players,
    'Avg Price': grouped['Price'].mean(),
    'Purchase Count': grouped['SN'].count(),
    'Total Value': grouped['Price'].sum(),
    'Relative Purchases': grouped['Price'].sum() / (grouped['SN'].nunique())
        }
grouped_stats = pd.DataFrame(columns)
grouped_stats = grouped_stats.round(2)
grouped_stats



,Avg Price,Proportion,Purchase Count,Relative Purchases,Total Value
Age Group,,,,,
06-10,2.76,0.07,5,2.76,13.82
10-14,2.99,0.04,3,2.99,8.96
14-18,2.76,0.15,11,2.76,30.41
18-22,3.02,0.26,20,3.18,60.34
22-26,2.94,0.28,23,3.22,67.61
26-30,2.69,0.05,4,2.69,10.77
30-34,2.03,0.05,5,2.54,10.17
34-38,3.74,0.07,5,3.74,18.72
38-42,3.65,0.03,2,3.65,7.30


# **Top Spenders**

* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  * SN
  * Purchase Count
  * Average Purchase Price
  * Total Purchase Value

In [14]:
top_spenders_group = raw_data.groupby('SN')

columns = {
    #'SN': top_spenders_group['SN'],
    'Purchase Count': top_spenders_group['Price'].count(),
    'Average Purchase Price': (top_spenders_group['Price'].sum() / top_spenders_group['Price'].count()).round(2),
    'Total Purchase Value': top_spenders_group['Price'].sum()
}
top_spenders_df = pd.DataFrame(columns)
top_spenders_df = top_spenders_df.sort_values(by=['Total Purchase Value'],ascending = False)
top_spenders_df.head(5)

,Average Purchase Price,Purchase Count,Total Purchase Value
SN,,,
Sundaky74,3.70,2,7.41
Aidaira26,2.56,2,5.13
Eusty71,4.81,1,4.81
Chanirra64,4.78,1,4.78
Alarap40,4.71,1,4.71


# **Most Popular Items**

* Identify the 5 most popular items by purchase count, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [17]:
items_group = raw_data.groupby(['Item ID','Item Name','Price'])

#print(items_group['Price'].sum())

columns = {
    'Purchase Count': items_group['Price'].count(),
    'Total Purchase Value': items_group['Price'].sum()
}
#columns

top_items_df = pd.DataFrame(columns)
top_items_df = top_items_df.sort_values('Purchase Count', ascending = False)
top_items_df.head(5)

,,,Purchase Count,Total Purchase Value
Item ID,Item Name,Price,,
94,Mourning Blade,3.64,3,10.92
90,Betrayer,4.12,2,8.24
111,Misery's End,1.79,2,3.58
64,Fusion Pummel,2.42,2,4.84
154,Feral Katana,4.11,2,8.22


# **Most Profitable Items**

* Identify the 5 most profitable items by total purchase value, then list (in a table):
  * Item ID
  * Item Name
  * Purchase Count
  * Item Price
  * Total Purchase Value

In [18]:
top_items_df = top_items_df.sort_values('Total Purchase Value', ascending = False)
top_items_df.head(5)

,,,Purchase Count,Total Purchase Value
Item ID,Item Name,Price,,
94,Mourning Blade,3.64,3,10.92
117,"Heartstriker, Legacy of the Light",4.71,2,9.42
93,Apocalyptic Battlescythe,4.49,2,8.98
90,Betrayer,4.12,2,8.24
154,Feral Katana,4.11,2,8.22
